# TensorFlow_DNNClassifier


In [30]:
import tensorflow as tf
from tensorflow import estimator 
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')

### Data Set

In [31]:
wine_data = load_wine()
print(wine_data.keys())
feat_data = wine_data['data']
labels = wine_data['target']

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])


### Split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,labels,test_size=0.33,random_state=77)

### Scale the Data

In [33]:
scaler = MinMaxScaler()
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)

### tensorflow

In [34]:
X_train.shape

(119, 13)

In [35]:
feat_cols = [tf.feature_column.numeric_column("x", shape=[13])]

In [36]:
deep_model_tensorflow = estimator.DNNClassifier(hidden_units=[13,13,13],
                            feature_columns=feat_cols,
                            n_classes=3,
                            optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01) )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/0z/17yw4wrd249060n99vqwx9sr0000gn/T/tmpa_ltbfg2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1194b05c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [37]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},y=y_train,shuffle=True,batch_size=10,num_epochs=5)

In [38]:
deep_model_tensorflow.train(input_fn=input_fn,steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/0z/17yw4wrd249060n99vqwx9sr0000gn/T/tmpa_ltbfg2/model.ckpt.
INFO:tensorflow:loss = 10.250168, step = 1
INFO:tensorflow:Saving checkpoints for 60 into /var/folders/0z/17yw4wrd249060n99vqwx9sr0000gn/T/tmpa_ltbfg2/model.ckpt.
INFO:tensorflow:Loss for final step: 3.8627694.


In [39]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},shuffle=False)

In [40]:
preds = list(deep_model_tensorflow.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/0z/17yw4wrd249060n99vqwx9sr0000gn/T/tmpa_ltbfg2/model.ckpt-60
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [41]:
predictions = [p['class_ids'][0] for p in preds]

In [42]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.74      1.00      0.85        17
          1       0.44      0.76      0.56        21
          2       0.00      0.00      0.00        21

avg / total       0.37      0.56      0.44        59

